In [2]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
import os

os.environ["AZURE_OPENAI_API_KEY"] = "api_key_here"
os.environ["AZURE_OPENAI_ENDPOINT"] = "api_endpoint_here"
os.environ["Azure_deployment"] = "deplyment_name_here"
os.environ["Azure_api_version"] = "api_version_here"
os.environ["POSTGRES_USERNAME"] = "postgres"
os.environ["POSTGRES_PASSWORD"] = "postgres_password_here"
DATABASE_URL = "database_url_here"
db = SQLDatabase.from_uri(DATABASE_URL)
examples = [
    {
        "input":"List all the employees who can work on front-end development",
        "query":"""SELECT employee_name,primary_skill,secondary_skill,tertiary_skill FROM employees WHERE primary_skill IN ('HTML', 'CSS', 'JavaScript', 'React.js', 'Angular.js', 'Vue.js') 
        OR secondary_skill IN ('HTML', 'CSS', 'JavaScript', 'React.js', 'Angular.js', 'Vue.js')
        OR tertiary_Skill IN ('HTML', 'CSS', 'JavaScript', 'React.js', 'Angular.js', 'Vue.js')
        OR Current_role IN ('Frontend Developer', 'UI Developer', 'Web Developer');
"""
    },
    
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    AzureOpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)
# DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.



If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


llm = AzureChatOpenAI(
    azure_deployment="deployment_name_here",
    api_version="2023-07-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
response = agent.invoke("Can you get me list of employees who are skilled in machine laearning and located in bangalore")
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


employees, employees_personal_details, people_table
Invoking: `sql_db_schema` with `{'table_names': 'employees, employees_personal_details'}`



CREATE TABLE employees (
	employee_id BIGINT NOT NULL, 
	employee_name TEXT, 
	grade TEXT, 
	organisation_unit TEXT, 
	curr_role TEXT, 
	currently_assigned_to_a_project TEXT, 
	currently_assigned_project_name TEXT, 
	current_assignment_end_date TEXT, 
	total_years_of_experience BIGINT, 
	employee_profile_summary TEXT, 
	primary_skill TEXT, 
	primary_skill_years_of_experience BIGINT, 
	secondary_skill TEXT, 
	secondary_skill_years_of_experience BIGINT, 
	tertiary_skill TEXT, 
	tertiary_skill_years_of_experience BIGINT, 
	industry_recognised_certifications TEXT, 
	specialized_knowledge TEXT, 
	education_qualification TEXT, 
	major TEXT, 
	current_performance_plan TEXT, 
	CONSTRAINT employees_pkey PRIMARY KEY (employee_id)
)

/*
3 rows from employees table:
e

In [ ]:
#python fucntion to parse a pdf document
